In [1]:
from replicate.data.loading import DataLoader
from replicate.data.analysis import train_test_split_data, analyze_train_test_results
from replicate.optimizers.mm import MMOptimizer

# Initialize loader and optimizer
loader = DataLoader()
optimizer = MMOptimizer(measure='ete')

sym = 'SP500'
START_DATE = '2010-01-01'
END_DATE = '2010-12-31'

data = train_test_split_data(
    loader=loader,
    data_type=DataLoader.SP500_2010,
    target_symbol=sym,
    train_pct=0.5,
    start_date=START_DATE,
    end_date=END_DATE,
    time_col='Date'
)

result_dict = optimizer.optimize(
    data['train']['X'], 
    data['train']['r'], 
    lambda_=1e-7, 
    u=0.025, 
    p_neg_exp=8, 
    thres=1e-3,
    hub=0.5
)

# Create equal weights
n_assets = len(result_dict['weights'])
equal_weights = np.ones(n_assets) / n_assets

# Debug the data
debug_data_scaling(data['train'], data['test'], equal_weights)

train_metrics, test_metrics = analyze_train_test_results(
    train_data=data['train'],
    test_data=data['test'],
    weights=random_weights,
    asset_names=data['asset_names'],
    title_prefix=f"{sym} Tracking",
    plot_combined=True  # This will use the new combined visualization
)

ValueError: Constraints impossible to satisfy: with 1 assets and max weight 0.025, maximum possible weight sum is 0.03. Either increase u to at least 1.000 or provide more assets.

In [13]:

import polars as pl

train_pct = 0.5
time_col = 'Date'

target, constituents = loader.load(
        data_type=DataLoader.SP500_2010
    )

n_samples = target.select(pl.len()).collect().item()
train_size = int(n_samples * train_pct)

# Get asset names before any filtering
asset_names = [c for c in constituents.collect_schema().names() if c != time_col]

# Create train/test splits using row numbers
with_row_nums = target.with_row_index("row_num")
train_filter = pl.col("row_num") < train_size
test_filter = pl.col("row_num") >= train_size

constituents_with_nums = constituents.with_row_index("row_num")
train_X = constituents_with_nums.filter(train_filter).drop(["row_num", time_col])
test_X = constituents_with_nums.filter(test_filter).drop(["row_num", time_col])

In [17]:
target.collect()

1436513D UN Equity,1500785D UN Equity,1518855D US Equity,9876566D UN Equity,A UN Equity,AA UN Equity,AAPL UW Equity,ABC UN Equity,ABT UN Equity,ADBE UW Equity,ADM UN Equity,ADP UW Equity,ADSK UW Equity,AEE UN Equity,AEP UN Equity,AES UN Equity,AET UN Equity,AFL UN Equity,AGN UN Equity,AIG UN Equity,AIV UN Equity,AIZ UN Equity,AKAM UW Equity,ALL UN Equity,ALTR UW Equity,AMAT UW Equity,AMGN UW Equity,AMP UN Equity,AMT UN Equity,AMZN UW Equity,AN UN Equity,ANTM UN Equity,AON UN Equity,APA UN Equity,APC UN Equity,APD UN Equity,APH UN Equity,…,UNH UN Equity,UNM UN Equity,UNP UN Equity,UPS UN Equity,USB UN Equity,UTX UN Equity,V UN Equity,VAR UN Equity,VFC UN Equity,VLO UN Equity,VMC UN Equity,VNO UN Equity,VRSN UW Equity,VTR UN Equity,VZ UN Equity,WAT UN Equity,WEC UN Equity,WFC UN Equity,WFM UW Equity,WHR UN Equity,WM UN Equity,WMB UN Equity,WMT UN Equity,WU UN Equity,WY UN Equity,WYN UN Equity,WYNN UW Equity,XEL UN Equity,XL UN Equity,XLNX UW Equity,XOM UN Equity,XRAY UW Equity,XRX UN Equity,YUM UN Equity,ZBH UN Equity,ZION UW Equity,Date
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
0.020559,-0.008016,0.01907,0.009762,0.007401,0.03288,0.015746,0.021479,0.008706,0.008428,0.005112,0.000234,0.010232,-0.006799,0.004311,0.02705,0.041037,0.02854,0.017167,-0.003004,0.002506,0.023064,0.023293,0.012317,0.01635,0.025823,0.020672,0.027287,0.005796,-0.004609,0.006266,0.025049,-0.010434,0.02617,0.037168,0.025908,-0.005199,…,0.034422,0.021512,0.023316,0.014124,0.016439,0.031983,0.007778,0.006829,0.001913,0.068054,0.001135,0.001144,0.020628,-0.008917,0.004523,-0.005326,-0.002008,0.012229,0.014574,0.015983,0.010351,0.025147,0.014593,0.012732,0.026424,0.005455,0.098403,-0.0066,0.021273,0.012768,0.014078,0.00455,0.020096,0.003432,0.015396,0.038969,"""2010-01-04"""
0.028056,-0.008081,0.007602,-0.016714,-0.010862,-0.031233,0.000934,-0.007134,-0.00808,0.016446,0.005401,-0.005371,-0.015193,-0.003962,-0.011451,-0.01024,-0.014259,0.029012,-0.009937,-0.018734,0.025067,0.029844,0.029707,0.016441,-0.012172,-0.007687,-0.009006,0.023324,0.01655,0.0059,0.015568,0.010888,-0.006326,0.011807,0.028113,-0.008298,-0.020896,…,-0.001563,0.014548,0.013916,0.001709,0.025347,-0.014929,-0.011461,0.014206,0.019077,0.025715,0.003805,-0.00757,0.01051,0.004843,0.001806,-0.01363,-0.013067,0.02745,-0.008259,-0.002066,-0.004392,0.012494,-0.009958,-0.004191,0.021228,0.021201,0.060819,-0.011862,-0.006945,-0.012607,0.003906,-0.01189,0.00116,-0.00342,0.031655,0.035256,"""2010-01-05"""
0.009101,0.005639,-0.003482,-0.014542,-0.003554,0.052076,-0.015907,-0.009453,0.005554,-0.002122,-0.002529,-0.002349,0.002373,-0.006873,0.010134,-0.010346,-0.004009,0.008783,0.003253,-0.006478,0.017725,-0.0029,-0.008243,0.000325,-0.011885,-0.00212,-0.0075,0.013966,0.005219,-0.018116,0.038324,0.016221,-0.000262,0.015683,0.008714,-0.008246,0.003781,…,0.009853,0.008897,0.001507,-0.007378,0.010658,-0.005249,-0.013429,0.006062,-0.004013,0.025071,0.017378,-0.004029,-0.010801,-0.011248,-0.028752,0.00181,0.010185,0.001427,0.011586,-0.008815,-0.000293,0.032452,-0.002234,-0.000527,-0.011057,0.018835,-0.013116,0.001923,-0.010215,-0.006784,0.008642,0.006589,-0.009259,-0.007148,-0.000323,0.086959,"""2010-01-06"""
0.000345,0.009348,-0.027373,0.028265,-0.001294,-0.021214,-0.001848,-0.016039,0.008284,-0.019405,-0.010457,-0.000471,0.00513,-0.009468,0.0086,0.000744,0.031798,0.010734,-0.007993,-0.019221,0.036638,0.01227,0.000378,0.005172,-0.004901,-0.010587,-0.009156,0.007502,-0.002708,-0.017013,0.017717,0.037141,-0.002908,-0.015166,-0.001639,-0.005747,-0.006645,…,0.038357,0.022047,-0.008885,-0.007605,0.019404,0.004273,0.009306,-0.016414,0.019337,0.007443,-0.012623,0.015461,-0.013748,0.011146,-0.005953,0.004269,-0.002018,0.036285,-0.004294,0.021468,0.002353,-0.007088,0.000